# Python Projet 5 - Prédiction des tags

## Multilabel classification avec la méthode One-vs-Rest

Dans le cadre du cours OpenClassrooms Machine Learning et du projet d'évaluation #5, ce notebook présente les différents modèles de prédiction des tags ainsi qu'une comparaison avec l'analyse non supervisée obtenue par LDA.

Author : Vincent Arrigoni, 04/2023

Regexp : https://www.debuggex.com/cheatsheet/regex/python

Tout savoir :https://datascientest.com/regex-tout-savoir

Test regexp : https://regex101.com/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# import of libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_score, recall_score, jaccard_score, classification_report
from gensim.models import Word2Vec
import re
from tqdm import tqdm
from time import time
from sklearn.utils import shuffle

# import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow_text as text
# from official.nlp import optimization  # to create AdamW optimizer

In [3]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Suppress specific warnings
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [4]:
data = pd.read_csv('/content/drive/MyDrive/OCS/Projet5/msg_preprocessed.csv')
# data = pd.read_csv('/content/drive/MyDrive/OCS/Projet5/msg_preprocessed_reduced.csv')
data.head(10)

,Unnamed: 0,Text,Tags,Token,Tags_lst,Tags_lst_new,Token_lst_new
0,0,how do I calculate a rolling idxmax<p>consider...,<python><pandas><numpy><dataframe><series>,"['calcul', 'roll', 'idxmax', 'consid', 'pd', '...","['python', 'pandas', 'numpy', 'dataframe', 'se...",['python'],"['calcul', 'roll', 'idxmax', 'consid', 'pd', '..."
1,1,Object-Oriented Callbacks for C++?<p>Is there ...,<c++><oop><callback><pointer-to-member><eiffel>,"['object', 'orient', 'callback', 'c++', 'libra...","['c++', 'oop', 'callback', 'pointer-to-member'...",['c++'],"['object', 'orient', 'callback', 'c++', 'libra..."
2,2,TDD and ADO.NET Entity Framework<p>I've been p...,<unit-testing><entity-framework><ado.net><tdd>...,"['tdd', 'ado', '.net', 'entiti', 'framework', ...","['unit-testing', 'entity-framework', 'ado.net'...",['unit-testing'],"['tdd', 'ado', '.net', 'entiti', 'framework', ..."
3,3,Better techniques for trimming leading zeros i...,<sql><sql-server><sql-server-2005><tsql><string>,"['better', 'techniqu', 'trim', 'lead', 'zero',...","['sql', 'sql-server', 'sql-server-2005', 'tsql...","['sql', 'sql-server', 'string']","['better', 'techniqu', 'trim', 'lead', 'zero',..."
4,4,"""No X11 DISPLAY variable"" - what does it mean?...",<java><linux><variables><x11><headless>,"['x11', 'display', 'variabl', 'mean', 'tri', '...","['java', 'linux', 'variables', 'x11', 'headless']","['java', 'linux']","['x11', 'display', 'variabl', 'mean', 'tri', '..."
5,5,Why doesn't .NET find the OpenSSL.NET dll?<p><...,<c#><.net><dll><dllimport><dllnotfoundexception>,"['.net', 'find', 'openssl', '.net', 'dll', 'ed...","['c#', '.net', 'dll', 'dllimport', 'dllnotfoun...","['c#', '.net']","['.net', 'find', 'openssl', '.net', 'dll', 'ed..."
6,6,Iterating through/Parsing JSON Object via Java...,<javascript><json><jquery><parsing><loops>,"['iter', 'pars', 'json', 'object', 'via', 'jav...","['javascript', 'json', 'jquery', 'parsing', 'l...","['javascript', 'json', 'jquery']","['iter', 'pars', 'json', 'object', 'via', 'jav..."
7,7,Class inherited from class without default con...,<c++><class><inheritance><constructor><default...,"['class', 'inherit', 'class', 'without', 'defa...","['c++', 'class', 'inheritance', 'constructor',...",['c++'],"['class', 'inherit', 'class', 'without', 'defa..."
8,8,libxml2 vs expat for an XMPP server<p>I'm tryi...,<c><go><xmpp><libxml2><expat-parser>,"['libxml2', 'vs', 'expat', 'xmpp', 'server', '...","['c', 'go', 'xmpp', 'libxml2', 'expat-parser']",['c'],"['libxml2', 'vs', 'expat', 'xmpp', 'server', '..."
9,9,Implications of deploying a Debug build of an ...,<c#><security><debugging><deployment><compiler...,"['implic', 'deploy', 'debug', 'build', 'applic...","['c#', 'security', 'debugging', 'deployment', ...","['c#', 'security', 'debugging']","['implic', 'deploy', 'debug', 'build', 'applic..."


In [5]:
def regexp_Token(x):
  return(re.findall(r'(?:C\+\+)|(?:c\+\+)|(?:c\#)|(?:C\#)|(?:\.net)|(?:\.NET)|\w{2,}', x['Token_lst_new']))

def regexp_Tags(x):
  return(re.findall(r'(?:C\+\+)|(?:c\+\+)|(?:c\#)|(?:C\#)|(?:\.net)|(?:\.NET)|\w{2,}', x['Tags_lst_new']))

ds = pd.DataFrame({'Token' : data[:5000].apply(regexp_Token, axis=1), 'Tags' : data[:5000].apply(regexp_Tags, axis=1)})
ds = shuffle(ds)
ds.reset_index(inplace=True)
ds.head(10)

,index,Token,Tags
0,60,"[effect, strategi, leav, audit, trail, chang, ...",[database]
1,1584,"[open, default, text, editor, linux, need, ope...",[linux]
2,3752,"[unexpect, credenti, type, none, expect, servi...","[python, django, python]"
3,4417,"[perform, impact, rdd, api, vs, udf, mix, data...",[performance]
4,3556,"[list, serial, com, port, window, look, robust...","[python, windows]"
5,3725,"[toggl, main, menu, visibl, use, alt, key, wpf...",[wpf]
6,4072,"[set, toolbar, item, uinavigationcontrol, ipho...","[cocoa, touch]"
7,2941,"[rampart, postdispatchverificationhandl, inval...",[java]
8,2862,"[uitapgesturerecogn, uiview, subview, respond,...","[ios, iphone, objective]"
9,2255,"[activemq, rabbitmq, zeromq, interest, hear, e...",[]


In [6]:
# Classifier choice
# Choose Logistic Regression
# Else is RandomForestClassifier
Clf_choice = True

## Approche Bag Of Words pour la classification par Régression Logistique

In [7]:
def dummy(doc):
  return doc
  # https://stackoverflow.com/questions/35867484/pass-tokens-to-countvectorizer
  # pas besoin que countvectorizer fasse un preprocessing ou une tokenization des documents

vectorizer = CountVectorizer(tokenizer=dummy, preprocessor=dummy, binary=True)
X = vectorizer.fit_transform(ds['Token'])
X = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

LabelBinarizer = MultiLabelBinarizer()
y = LabelBinarizer.fit_transform(ds['Tags'])
y = pd.DataFrame(y, columns=LabelBinarizer.classes_)

In [8]:
# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X, y)

params_LR = {'C': 0.5, 'max_iter': 500, 'penalty': 'none', 'solver': 'lbfgs'}
params_RF = {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': None, 'bootstrap': False}

# Build the model
if Clf_choice:
  model = MultiOutputClassifier(LogisticRegression(**params_LR))
  # https://scikit-learn.org/stable/modules/multiclass.html
else:
  model = MultiOutputClassifier(RandomForestClassifier(**params_RF))

In [9]:
# search = GridSearchCV(
#     estimator = model,
#     param_grid={'estimator__C':np.logspace(-3, 0, 4)},
#     scoring = ['accuracy', 'precision_micro', 'recall_micro'],
#     refit = 'precision_micro',
#     cv = 5
# ).fit(X_train, y_train)
# # https://datascience.stackexchange.com/questions/107867/how-to-train-multioutput-classification-with-hyperparameter-tuning-in-sklearn
# # https://stackoverflow.com/questions/41899132/invalid-parameter-for-sklearn-estimator-pipeline
# # https://scikit-learn.org/stable/modules/model_evaluation.html#from-binary-to-multiclass-and-multilabel

# # error : Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

# print(search.best_estimator_)
# print(search.best_params_)
# clf_BOW = search.best_estimator_

In [10]:
model.fit(X_train, y_train)
clf_BOW = model

In [11]:
y_pred = clf_BOW.predict(X_test)
print("Classifier report:\n", classification_report(y_test, y_pred, target_names=LabelBinarizer.classes_, zero_division=0))
# Calculate the Jaccard score
score = jaccard_score(y_test, y_pred, average='micro')
print(f"Jaccard score: {score:.4f}")

Classifier report:
                 precision    recall  f1-score   support

          .net       0.65      0.51      0.57       157
          ajax       0.60      0.20      0.30        15
     algorithm       0.50      0.31      0.38        16
       android       0.84      0.67      0.75        57
        arrays       0.60      0.43      0.50        21
           asp       0.75      0.45      0.56        53
          bash       1.00      0.31      0.47        13
            c#       0.67      0.63      0.65       180
           c++       0.76      0.66      0.71       122
         cocoa       0.67      0.25      0.36        16
           css       0.53      0.35      0.42        26
      database       0.00      0.00      0.00        22
     debugging       0.33      0.08      0.13        12
        django       0.93      0.72      0.81        18
           gcc       0.40      0.12      0.19        16
     hibernate       0.80      0.17      0.28        24
          html       0.43  

## Approche TF-IDF pour la classification par Régression Logistique

In [12]:
def dummy(doc):
  return doc

# Binarized before Train/Test split
LabelBinarizer = MultiLabelBinarizer()
y_tfidf = LabelBinarizer.fit_transform(ds['Tags'])
y_tfidf = pd.DataFrame(y_tfidf, columns=LabelBinarizer.classes_)

# Train-Test split
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(ds['Token'], y_tfidf)

# Binarized after Train/Test split as the frequency depends on the sample size
# Perform tf-idf on all dataset could induce data leak
tfidf_vectorizer = TfidfVectorizer(preprocessor=dummy, tokenizer=dummy)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_tfidf)
X_test_tfidf = tfidf_vectorizer.transform(X_test_tfidf)
# https://www.kaggle.com/code/neerajmohan/nlp-text-classification-using-tf-idf-features

params_LR = {'C': 0.5, 'max_iter': 500, 'penalty': 'none', 'solver': 'lbfgs'}
params_RF = {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': None, 'bootstrap': False}

# Build the model
if Clf_choice:
  model = MultiOutputClassifier(LogisticRegression(**params_LR))
  # https://scikit-learn.org/stable/modules/multiclass.html
else:
  model = MultiOutputClassifier(RandomForestClassifier(**params_RF))

In [13]:
# search_tfidf = GridSearchCV(
#     estimator = model,
#     param_grid={'estimator__C':np.logspace(-3, 0, 4)},
#     scoring = ['accuracy', 'precision_micro', 'recall_micro'],
#     refit = 'precision_micro',
#     cv = 5
# ).fit(X_train_tfidf, y_train_tfidf)

# print(search_tfidf.best_estimator_)
# print(search_tfidf.best_params_)
# clf_TFIDF = search_tfidf.best_estimator_

In [14]:
model.fit(X_train_tfidf, y_train_tfidf)
clf_TFIDF = model

In [15]:
y_pred_tfidf = clf_TFIDF.predict(X_test_tfidf)
print("Classifier report:\n", classification_report(y_test_tfidf, y_pred_tfidf, target_names=LabelBinarizer.classes_, zero_division=0))
# Calculate the Jaccard score
score = jaccard_score(y_test_tfidf, y_pred_tfidf, average='micro')
print(f"Jaccard score: {score:.4f}")

Classifier report:
                 precision    recall  f1-score   support

          .net       0.56      0.44      0.49       155
          ajax       0.86      0.38      0.52        16
     algorithm       0.83      0.21      0.33        24
       android       0.93      0.69      0.79        55
        arrays       0.78      0.37      0.50        19
           asp       0.63      0.44      0.52        50
          bash       0.60      0.18      0.27        17
            c#       0.77      0.58      0.66       193
           c++       0.86      0.67      0.75        90
         cocoa       0.40      0.10      0.16        20
           css       0.52      0.42      0.47        26
      database       0.40      0.16      0.23        25
     debugging       1.00      0.33      0.50        18
        django       0.94      0.76      0.84        21
           gcc       0.50      0.14      0.22        14
     hibernate       0.82      0.69      0.75        13
          html       0.69  

## Word Embeddings with Word2Vec and Logistic Regression

In [16]:
# Word2Vec

# Binarized before Train/Test split
LabelBinarizer = MultiLabelBinarizer()
y_wrd2vec = LabelBinarizer.fit_transform(ds['Tags'])
# y_wrd2vec = pd.DataFrame(y_wrd2vec, columns=LabelBinarizer.classes_)

# Train-Test split
X_train_wrd2vec, X_test_wrd2vec, y_train_wrd2vec, y_test_wrd2vec = train_test_split(ds['Token'], y_wrd2vec)

# Train the word2vec model
w2v_model = Word2Vec(X_train_wrd2vec,
                     vector_size=100,
                     window=5,
                     min_count=5)

In [17]:
len(w2v_model.wv.index_to_key) # all the words that appear at least twice in the document

7447

In [18]:
w2v_model.wv[1] # embedding of the second word of the first sentence

array([-1.5380335 ,  1.3520051 ,  0.68525785,  0.1333693 , -0.23950645,
       -1.6950651 , -0.07335615,  1.6979914 , -0.76676244, -1.4604728 ,
        0.4830579 , -3.3039083 , -0.12035044,  1.1960665 ,  0.34776387,
        0.49385056,  2.7298424 ,  1.2952478 , -0.4758016 , -0.46781862,
        1.7098095 , -0.5747277 ,  2.6767373 , -3.6632836 ,  0.44861346,
        1.0962154 , -1.117364  ,  0.02052526, -0.7121209 ,  1.6273644 ,
        2.5384274 ,  0.5348175 ,  0.9884123 , -0.91006327,  0.15834996,
        0.90636545,  0.96163833, -1.022436  , -0.1871068 ,  0.48875907,
        1.9405227 , -1.5439018 , -1.8450899 , -0.43729022, -0.27932653,
        1.2033367 ,  1.018245  , -0.21056052,  0.49831614,  1.6900855 ,
        1.9460907 , -1.9822507 ,  1.124801  ,  0.7199742 , -0.9211783 ,
        2.4180799 ,  0.9552146 , -0.7043153 , -1.5126437 ,  2.0663135 ,
       -2.076018  ,  0.22789106, -1.1281697 , -0.3512486 , -0.16580963,
        0.85600257, -0.42602852, -0.15646937,  0.7811515 ,  3.09

In [19]:
w2v_model.wv.most_similar(positive=["python"])

[('setup', 0.8972122669219971),
 ('tkinter', 0.8929158449172974),
 ('packag', 0.8770855665206909),
 ('rubi', 0.85679692029953),
 ('pip', 0.8475775718688965),
 ('code_pars', 0.8363302946090698),
 ('librari', 0.8304653167724609),
 ('nuget', 0.8293452262878418),
 ('chai', 0.8279942274093628),
 ('msi', 0.8264946937561035)]

In [20]:
words = set(w2v_model.wv.index_to_key) # vocabulary correspondance
# https://medium.com/@dilip.voleti/classification-using-word2vec-b1d79d375381

X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in X_train_wrd2vec]) # concatenate word embeddings of the same sentence in the same element of X_train_vect

X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])  for ls in X_test_wrd2vec])

In [21]:
# Compute sentence embeddings by averaging the word embeddings for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [22]:
params_LR = {'C': 0.5, 'max_iter': 500, 'penalty': 'none', 'solver': 'lbfgs'}
params_RF = {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': None, 'bootstrap': False}

# Build the model
if Clf_choice:
  model_WRD2VEC = MultiOutputClassifier(LogisticRegression(**params_LR))
  # https://scikit-learn.org/stable/modules/multiclass.html
else:
  model_WRD2VEC = MultiOutputClassifier(RandomForestClassifier(**params_RF))

model_WRD2VEC.fit(X_train_vect_avg, y_train_wrd2vec)

MultiOutputClassifier(estimator=LogisticRegression(C=0.5, max_iter=500,
                                                   penalty='none'))

In [25]:
y_pred_wrd2vec = model_WRD2VEC.predict(X_test_vect_avg)
print("Classifier report:\n", classification_report(y_test_wrd2vec, y_pred_wrd2vec, target_names=LabelBinarizer.classes_, zero_division=0))
# Calculate the Jaccard score
score = jaccard_score(y_test_wrd2vec, y_pred_wrd2vec, average='samples')
print(f"Jaccard score: {score:.4f}")

Classifier report:
                 precision    recall  f1-score   support

          .net       0.65      0.44      0.53       149
          ajax       0.50      0.33      0.40        18
     algorithm       0.16      0.19      0.17        16
       android       0.82      0.59      0.69        61
        arrays       0.38      0.29      0.33        17
           asp       0.56      0.36      0.44        50
          bash       0.35      0.50      0.41        12
            c#       0.64      0.39      0.48       194
           c++       0.76      0.50      0.60       113
         cocoa       0.43      0.19      0.26        16
           css       0.44      0.40      0.42        30
      database       0.30      0.11      0.16        27
     debugging       0.14      0.07      0.10        14
        django       0.46      0.25      0.32        24
           gcc       0.12      0.30      0.18        10
     hibernate       0.33      0.25      0.29        12
          html       0.56  

## BERT NLP algorithm

In [26]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', truncation=True, padding=True)
model_BERT = TFBertModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [28]:
# ID ENCODING BY HAND
def bert_encode(token_lst, tokenizer, max_len=64):
  all_tokens = []
  all_masks = []
  all_segments = []

  for text in token_lst:

    text = text[:max_len-2] # How to deal with truncation ?
    text = ["[CLS]"] + text + ["[SEP]"] # special tokens
    pad_len = max_len - len(text) # necesary PAD length

    token_id = tokenizer.convert_tokens_to_ids(text) + [0] * pad_len
    pad_masks = [1] * len(text) + [0] * pad_len
    segment_ids = [0] * max_len

    all_tokens.append(token_id)
    all_masks.append(pad_masks)
    all_segments.append(segment_ids)

  return {'input_ids' : np.array(all_tokens),
          'token_type_ids' : np.array(all_masks),
          'attention_mask' : np.array(all_segments)}

emb_BERT = np.array([])
batch_size=500
for i in tqdm(range(len(ds)//batch_size)):
  # Batch it because otherwise too big for the model to train on
  encoded_tokens = bert_encode(ds.loc[i*batch_size:(i+1)*batch_size-1,'Token'], tokenizer)
  # print(encoded_tokens)
  # print(len(encoded_tokens))
  embeddings = model_BERT(encoded_tokens)
  if len(emb_BERT)==0:
    emb_BERT = np.array(embeddings['pooler_output'])
  else:
    emb_BERT = np.vstack((emb_BERT, np.array(embeddings['pooler_output'])))
    # De-batching it to usual format

100%|██████████| 10/10 [00:06<00:00,  1.61it/s]


In [29]:
print(len(embeddings['pooler_output']))

500


In [30]:
x = emb_BERT
y = ds['Tags']

# Initialize MultiLabelBinarizer
LabelBinarizer = MultiLabelBinarizer()

y = LabelBinarizer.fit_transform(y)

X_train_BERT, X_test_BERT, y_train_BERT, y_test_BERT = train_test_split(x, y, test_size=0.2, random_state=42)

params_LR = {'C': 0.5, 'max_iter': 500, 'penalty': 'none', 'solver': 'lbfgs'}
params_RF = {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': None, 'bootstrap': False}

# Build the model
if Clf_choice:
  clf_BERT = MultiOutputClassifier(LogisticRegression(**params_LR))
  # https://scikit-learn.org/stable/modules/multiclass.html
else:
  clf_BERT = MultiOutputClassifier(RandomForestClassifier(**params_RF))

In [31]:
clf_BERT.fit(X_train_BERT, y_train_BERT)

MultiOutputClassifier(estimator=LogisticRegression(C=0.5, max_iter=500,
                                                   penalty='none'))

In [32]:
# Make predictions on the testing data
y_pred_BERT = clf_BERT.predict(X_test_BERT)

# Evaluate the classifier using classification_report
print("Classifier report:\n", classification_report(y_test_BERT, y_pred_BERT, target_names=LabelBinarizer.classes_, zero_division=0))

# Calculate the Jaccard score
score = jaccard_score(y_test_BERT, y_pred_BERT, average='samples')
print(f"Jaccard score: {score:.4f}")

# Reset warnings to default
# warnings.resetwarnings()

Classifier report:
                 precision    recall  f1-score   support

          .net       0.30      0.22      0.25       123
          ajax       0.14      0.09      0.11        11
     algorithm       0.15      0.15      0.15        13
       android       0.54      0.59      0.57        44
        arrays       0.45      0.29      0.36        17
           asp       0.15      0.15      0.15        40
          bash       0.00      0.00      0.00         9
            c#       0.31      0.23      0.27       142
           c++       0.33      0.21      0.26        85
         cocoa       0.08      0.12      0.09        17
           css       0.13      0.24      0.17        17
      database       0.15      0.22      0.18        18
     debugging       0.00      0.00      0.00        11
        django       0.00      0.00      0.00        16
           gcc       0.14      0.07      0.09        15
     hibernate       0.00      0.00      0.00        12
          html       0.16  

In [33]:
# OVERALL ENCODING
def BERT_encode(batch):
  batch = list(map(lambda x : ' '.join(x) ,batch))
  encoded_input = tokenizer(batch, return_tensors='tf', truncation=True, padding=True)
  # grouped_inputs = {'input_ids':tf.cast(encoded_input.input_ids, tf.int16),
  #                   'token_type_ids' : tf.cast(encoded_input.token_type_ids, tf.int16),
  #                   'attention_mask' : tf.cast(encoded_input.attention_mask, tf.int16)}
  # print(tf.cast(encoded_input.input_ids, tf.int16))
  # # print(encoded_input.astype(int16))
  # print(len(encoded_input))
  return model_BERT(encoded_input)

emb_BERT = np.array([])
# Batch the dataset to compute embeddings more easily
batch_size = 100
max_len = 64 # max number of word to avoid Out Of Memory issue
nb_batches = len(ds)//batch_size
for i in tqdm(range(len(ds)//batch_size)):
  # Batch it because otherwise too big for the model to train on
  embeddings = BERT_encode(ds.loc[i*batch_size:(i+1)*batch_size-1,'Token'].str.slice(0,max_len))
  # embeddings = model_USE(encoded_tokens)
  if emb_BERT.size==0:
    emb_BERT = np.array(embeddings['pooler_output'])
    # print(emb_BERT.size)
  else:
    emb_BERT = np.vstack((emb_BERT, np.array(embeddings['pooler_output'])))
    # print('merging')
    # print(i)
    # print(emb_BERT.size)
    # De-batching it to usual format

100%|██████████| 50/50 [00:24<00:00,  2.06it/s]


In [34]:
x = emb_BERT
y = ds['Tags']

# Initialize MultiLabelBinarizer
LabelBinarizer = MultiLabelBinarizer()

y = LabelBinarizer.fit_transform(y)

X_train_BERT, X_test_BERT, y_train_BERT, y_test_BERT = train_test_split(x, y, test_size=0.2, random_state=42)

params_LR = {'C': 0.5, 'max_iter': 500, 'penalty': 'none', 'solver': 'lbfgs'}
params_RF = {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': None, 'bootstrap': False}

# Build the model
if Clf_choice:
  clf_BERT2 = MultiOutputClassifier(LogisticRegression(**params_LR))
  # https://scikit-learn.org/stable/modules/multiclass.html
else:
  clf_BERT2 = MultiOutputClassifier(RandomForestClassifier(**params_RF))

In [35]:
clf_BERT2.fit(X_train_BERT, y_train_BERT)

MultiOutputClassifier(estimator=LogisticRegression(C=0.5, max_iter=500,
                                                   penalty='none'))

In [36]:
# Make predictions on the testing data
y_pred_BERT2 = clf_BERT2.predict(X_test_BERT)

# Evaluate the classifier using classification_report
print("Classifier report:\n", classification_report(y_test_BERT, y_pred_BERT2, target_names=LabelBinarizer.classes_, zero_division=0))

# Calculate the Jaccard score
score = jaccard_score(y_test_BERT, y_pred_BERT, average='samples')
print(f"Jaccard score: {score:.4f}")

# Reset warnings to default
# warnings.resetwarnings()

Classifier report:
                 precision    recall  f1-score   support

          .net       0.52      0.40      0.45       123
          ajax       0.23      0.27      0.25        11
     algorithm       0.31      0.31      0.31        13
       android       0.58      0.57      0.57        44
        arrays       0.27      0.24      0.25        17
           asp       0.28      0.33      0.30        40
          bash       0.26      0.67      0.38         9
            c#       0.46      0.37      0.41       142
           c++       0.48      0.51      0.49        85
         cocoa       0.06      0.06      0.06        17
           css       0.05      0.06      0.05        17
      database       0.15      0.17      0.16        18
     debugging       0.14      0.18      0.16        11
        django       0.31      0.25      0.28        16
           gcc       0.20      0.13      0.16        15
     hibernate       0.54      0.58      0.56        12
          html       0.16  

## USE NLP Algorithm

In [37]:
import tensorflow_hub as hub

model_USE = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

In [38]:
def USE_encode(batch):
  batch = list(map(lambda x : ' '.join(x) ,batch))
  return model_USE(batch).numpy()

emb_USE = np.array([])
# Batch the dataset to compute embeddings more easily
batch_size = 200
nb_batches = len(ds)//batch_size
for i in range(len(ds)//batch_size):
  # Batch it because otherwise too big for the model to train on
  embeddings = USE_encode(ds.loc[i*batch_size:(i+1)*batch_size-1,'Token'])
  # embeddings = model_USE(encoded_tokens)
  if len(emb_USE)==0:
    emb_USE = np.array(embeddings)
  else:
    emb_USE = np.vstack((emb_USE, np.array(embeddings)))
    # De-batching it to usual format

In [39]:
embeddings.shape

(200, 512)

In [40]:
emb_USE.shape

(5000, 512)

In [41]:
x = emb_USE
y = ds['Tags']

# Initialize MultiLabelBinarizer
LabelBinarizer = MultiLabelBinarizer()

y = LabelBinarizer.fit_transform(y)

X_train_USE, X_test_USE, y_train_USE, y_test_USE = train_test_split(x, y, test_size=0.2, random_state=42)

params_LR = {'C': 0.5, 'max_iter': 500, 'penalty': 'none', 'solver': 'lbfgs'}
params_RF = {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': None, 'bootstrap': False}

# Build the model
if Clf_choice:
  USE_clf = MultiOutputClassifier(LogisticRegression(**params_LR))
  # https://scikit-learn.org/stable/modules/multiclass.html
else:
  USE_clf = MultiOutputClassifier(RandomForestClassifier(**params_RF))

In [42]:
USE_clf.fit(X_train_USE, y_train_USE)

MultiOutputClassifier(estimator=LogisticRegression(C=0.5, max_iter=500,
                                                   penalty='none'))

In [43]:
# Make predictions on the testing data
y_pred_USE = USE_clf.predict(X_test_USE)

# Evaluate the classifier using classification_report
print("Classifier report:\n", classification_report(y_test_USE, y_pred_USE, target_names=LabelBinarizer.classes_, zero_division=0))

# Calculate the Jaccard score
score = jaccard_score(y_test_USE, y_pred_USE, average='samples')
print(f"Jaccard score: {score:.4f}")

# Reset warnings to default
warnings.resetwarnings()

Classifier report:
                 precision    recall  f1-score   support

          .net       0.50      0.50      0.50       123
          ajax       0.50      0.27      0.35        11
     algorithm       0.35      0.46      0.40        13
       android       0.81      0.77      0.79        44
        arrays       0.54      0.41      0.47        17
           asp       0.41      0.53      0.46        40
          bash       0.17      0.22      0.19         9
            c#       0.54      0.55      0.54       142
           c++       0.58      0.72      0.64        85
         cocoa       0.45      0.29      0.36        17
           css       0.54      0.41      0.47        17
      database       0.19      0.33      0.24        18
     debugging       0.36      0.36      0.36        11
        django       0.70      0.88      0.78        16
           gcc       0.55      0.40      0.46        15
     hibernate       1.00      0.33      0.50        12
          html       0.49  

In [44]:
# import joblib

# with open('USE_clf.pkl', 'wb') as f:
#   joblib.dump((LabelBinarizer, USE_clf), f)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
import joblib
yolo, lolilol = joblib.load('USE_clf.pkl')